In [106]:
%matplotlib inline
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder, Imputer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, RandomForestClassifier
from sklearn.metrics import recall_score, precision_score, confusion_matrix
from sklearn.pipeline import Pipeline
import pickle
from imblearn.combine import SMOTEENN

## Read in data and set up DataFrame.

In [2]:
survey_place = pd.read_csv('data/caltrans_full_survey/survey_place.csv')

/home/g49/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (22,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
columns = ['arr_time', 'dep_time', 'trip_distance_miles', 'air_trip_distance_miles', 'prev_trip_duration_min', 'act_dur', 'act_cnt', 'tract_id', 'county_id', 'state_id', 'mode']
survey_place_less = survey_place[columns]

In [4]:
survey_place_less['arr_time'] = pd.to_datetime(survey_place_less['arr_time'], infer_datetime_format=True, format='%H:%M:%S').dt.hour
survey_place_less['dep_time'] = pd.to_datetime(survey_place_less['dep_time'], infer_datetime_format=True, format='%H:%M:%S').dt.hour

/home/g49/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/g49/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


## Determine percent missing.
## Drop rows where "mode" is missing.
## Impute missing data.

In [5]:
def get_percentage_missing(series):
   ''' Calculates percentage of NaN values in DataFrame
   :param series: Pandas DataFrame object
   :return: float
   '''
   num = series.isnull().sum()
   den = len(series)
   return round(num/den, 7)

In [6]:
get_percentage_missing(survey_place_less)

arr_time                   0.000002
dep_time                   0.000002
trip_distance_miles        0.241332
air_trip_distance_miles    0.236207
prev_trip_duration_min     0.236207
act_dur                    0.000002
act_cnt                    0.000004
tract_id                   0.000011
county_id                  0.000011
state_id                   0.000002
mode                       0.236207
dtype: float64

In [7]:
survey_place_less.dropna(subset=['mode'], inplace=True)

/home/g49/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
get_percentage_missing(survey_place_less)

arr_time                   0.000000
dep_time                   0.000000
trip_distance_miles        0.006709
air_trip_distance_miles    0.000000
prev_trip_duration_min     0.000000
act_dur                    0.000000
act_cnt                    0.000003
tract_id                   0.000011
county_id                  0.000011
state_id                   0.000000
mode                       0.000000
dtype: float64

In [9]:
for col in survey_place_less:
    survey_place_less[col].fillna((survey_place_less[col].mean()), inplace=True)

/home/g49/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [10]:
get_percentage_missing(survey_place_less)

arr_time                   0.0
dep_time                   0.0
trip_distance_miles        0.0
air_trip_distance_miles    0.0
prev_trip_duration_min     0.0
act_dur                    0.0
act_cnt                    0.0
tract_id                   0.0
county_id                  0.0
state_id                   0.0
mode                       0.0
dtype: float64

## Train-test split
## Map y to "transit" (1) and "not transit" (0)

In [11]:
y = survey_place_less['mode']

In [12]:
# X = survey_place_less[['arr_time', 'dep_time', 'trip_distance_miles', 'air_trip_distance_miles', 'prev_trip_duration_min', 'act_dur', 'act_cnt', 'tract_id', 'county_id', 'state_id']]
X = survey_place_less[['arr_time', 'dep_time', 'trip_distance_miles', 'air_trip_distance_miles', 'prev_trip_duration_min', 'act_dur', 'act_cnt', 'state_id']]

In [ ]:
X, y = SMOTEENN().fit_sample(X, y)

In [13]:
dict_not_transit = dict((key, 0) for key in range(15))
dict_transit = dict((key, 1) for key in range(15,30))
mode_dict = {**dict_not_transit, **dict_transit}

In [14]:
ymap = y.map(mode_dict)

In [15]:
print(f"{ymap[ymap>0].count()} of {len(ymap)} ({ymap[ymap>0].count()/len(ymap)*100:.2f}%) are public transit.")

12928 of 351745 (3.68%) are public transit.


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, ymap)
# X_train, X_test, y_train, y_test = train_test_split(X, y)

In [17]:
gbc = GradientBoostingClassifier()
gbc.fit(X_train, y_train);

In [18]:
y_pred_gbc = gbc.predict(X_test)

In [19]:
abc = AdaBoostClassifier()
abc.fit(X_train, y_train);

In [20]:
y_pred_abc = abc.predict(X_test)

In [21]:
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train);

In [22]:
y_pred_rfc = rfc.predict(X_test)

In [23]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [24]:
gbc_scaled = GradientBoostingClassifier()
gbc_scaled.fit(X_train_scaled, y_train);

In [25]:
y_pred_gbc_scaled = gbc_scaled.predict(X_test_scaled)

In [26]:
abc_scaled = AdaBoostClassifier()
abc_scaled.fit(X_train_scaled, y_train);

In [27]:
y_pred_abc_scaled = abc_scaled.predict(X_test_scaled)

In [28]:
rfc_scaled = RandomForestClassifier()
rfc_scaled.fit(X_train_scaled, y_train);

In [29]:
y_pred_rfc_scaled = rfc_scaled.predict(X_test_scaled)

In [81]:
gbcgrid = GridSearchCV(gbc, param_grid={'n_estimators':[400, 1000], 'max_depth':[3], 'learning_rate':[1]}, scoring='recall', n_jobs=-1, refit=True)

In [82]:
gbcgrid.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [400, 1000], 'max_depth': [3], 'learning_rate': [1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='recall', verbose=0)

In [83]:
gbcgrid.best_params_

{'learning_rate': 1, 'max_depth': 3, 'n_estimators': 1000}

In [90]:
abcgrid = GridSearchCV(abc, param_grid={'n_estimators':[10, 50, 100], 'learning_rate':[0.1, 1, 10]}, scoring='f1', n_jobs=-1, refit=True)

In [95]:
abcgrid.fit(X_train, y_train)

/home/g49/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/g49/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/g49/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/g49/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/g49/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classifi

GridSearchCV(cv=None, error_score='raise',
       estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [10, 50, 100], 'learning_rate': [0.1, 1, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='f1', verbose=0)

In [96]:
abcgrid.best_params_

{'learning_rate': 1, 'n_estimators': 10}

In [78]:
# rfcgrid = GridSearchCV(rfc, param_grid={'n_estimators':[75, 100, 125], 'max_depth':[4,5,6,7,8,9], 'class_weight':['balanced_subsample']}, scoring='recall', n_jobs=-1, refit=True)

In [79]:
# rfcgrid.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [75, 100, 125], 'max_depth': [4, 5, 6, 7, 8, 9], 'class_weight': ['balanced_subsample']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='recall', verbose=0)

In [80]:
# rfcgrid.best_params_

{'class_weight': 'balanced_subsample', 'max_depth': 8, 'n_estimators': 100}

In [84]:
for model in [gbcgrid, adagrid, rfcgrid]:
    print("------------------------------")
    print("------------------------------")
    print("------------------------------")
    print(model)
    y_pred = model.predict(X_test)
    print("\t------------------------------")
    print(f"\tPrecision score: {precision_score(y_test, y_pred):.3f}")
    print(f"\tRecall score: {recall_score(y_test, y_pred):.3f}")
    print("\t------------------------------")
    print("\tMost important features:\n")
#     for imp, feat in sorted(zip(model.feature_importances_, survey_place_less.columns), reverse=True):
#         print(f"\t{feat}: {imp:.3f}")
# for model in [gbc_scaled, abc_scaled, rfc_scaled]:
#     print("------------------------------")
#     print("------------------------------")
#     print("------------------------------")
#     print(model)
#     y_pred = model.predict(X_test_scaled)
#     print("\t------------------------------")
#     print(f"\tRecall score: {recall_score(y_test, y_pred):.3f}")
#     print("\t------------------------------")
#     print("\tMost important features:\n")
#     for imp, feat in sorted(zip(model.feature_importances_, survey_place_less.columns), reverse=True):
#         print(f"\t{feat}: {imp:.3f}")

------------------------------
------------------------------
------------------------------
GridSearchCV(cv=None, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [400, 1000], 'max_depth': [3], 'learning_rate': [1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='recall', verbose=0)
	------------------------------
	Precision score: 0.425
	Recall score: 0.566
	------------------------------
	Most important features:



In [105]:
pickle.dump(gbcgrid, open('gbc.pkl', 'wb'))